# MLlib_RDD_Based

In [1]:
import findspark
findspark.init()

## Thêm các thư viện cần thiết

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.mllib.classification import LabeledPoint, LogisticRegressionWithLBFGS
from pyspark.mllib.linalg import Vectors
import os
import sys

os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable

## Tạo session

In [3]:
spark=SparkSession.builder.master("local")\
    .appName("mllib_rdd_based")\
    .config("spark.some.config.option", "some-value")\
    .getOrCreate()

## Đọc dữ liệu

In [4]:
lines=spark.sparkContext.textFile("hdfs://localhost:9000/lab03/creditcard.csv")
lines.count()

284808

In [5]:
header=lines.first()

In [6]:
raw_data=lines.filter(lambda line: line !=header)
data=raw_data.map(lambda lines: lines.split(","))\
            .map(lambda line: tuple([float(x) for x in line[0:-1]]+[float(line[-1].strip('"'))]))

In [7]:
for row in data.take(5):
    print(row)

(0.0, -1.3598071336738, -0.0727811733098497, 2.53634673796914, 1.37815522427443, -0.338320769942518, 0.462387777762292, 0.239598554061257, 0.0986979012610507, 0.363786969611213, 0.0907941719789316, -0.551599533260813, -0.617800855762348, -0.991389847235408, -0.311169353699879, 1.46817697209427, -0.470400525259478, 0.207971241929242, 0.0257905801985591, 0.403992960255733, 0.251412098239705, -0.018306777944153, 0.277837575558899, -0.110473910188767, 0.0669280749146731, 0.128539358273528, -0.189114843888824, 0.133558376740387, -0.0210530534538215, 149.62, 0.0)
(0.0, 1.19185711131486, 0.26615071205963, 0.16648011335321, 0.448154078460911, 0.0600176492822243, -0.0823608088155687, -0.0788029833323113, 0.0851016549148104, -0.255425128109186, -0.166974414004614, 1.61272666105479, 1.06523531137287, 0.48909501589608, -0.143772296441519, 0.635558093258208, 0.463917041022171, -0.114804663102346, -0.183361270123994, -0.145783041325259, -0.0690831352230203, -0.225775248033138, -0.638671952771851, 0.

## Tiền xử lý dữ liệu

### Loại bỏ dữ liệu lặp

In [8]:
data.count()  

284807

In [9]:
data=data.distinct()
data.count()

283726

### Loại bỏ dữ liệu thiếu

In [10]:
data = data.filter(lambda line: all(x is not None and x != '' for x in line))

In [11]:
data.count()

283726

## Chuẩn hóa dữ liệu Min-Max

In [22]:
features = data.map(lambda line: line[0:-1])

min_features=features.reduce(lambda x, y: [min(x[i], y[i]) for i in range(len(x))])
max_features=features.reduce(lambda x, y: [max(x[i], y[i]) for i in range(len(x))])

normalized_data = data.map(lambda line: tuple([(line[i] - min_features[i]) / (max_features[i] - min_features[i]) for i in range(len(line)-1)]+[line[-1]]))

for row in normalized_data.take(5):
    print(row)

(0.0, 0.978541954971695, 0.7700666508227654, 0.8402984903939014, 0.2717964907547009, 0.7661203363388934, 0.2621916978704357, 0.26487543874149616, 0.7862983529047245, 0.4539809683822362, 0.5052673462220311, 0.38118772246581145, 0.7443415693042709, 0.48619017593610825, 0.6412190072734594, 0.3838396643725496, 0.46410517798669204, 0.7277939830919614, 0.6406810941347436, 0.5519304220394026, 0.5795297525747019, 0.5578399149746115, 0.4802369598542956, 0.6669378230986288, 0.33643999609599445, 0.5872902523783181, 0.446012969158175, 0.4163451447884128, 0.31342266347556097, 0.00010470527605604418, 0.0)
(5.787304967822585e-06, 0.9352170233299468, 0.7531176669488862, 0.8681408192619086, 0.26876550734448534, 0.7623287857209992, 0.28112212055047436, 0.27017718255653134, 0.788042262834494, 0.41060274137949276, 0.5130180380913919, 0.32242211351494776, 0.7066833600612961, 0.5038542274352835, 0.6404734520442342, 0.5116969543365681, 0.3574426288295902, 0.7633809907036572, 0.644945381986696, 0.386683126520

## Tách dữ liệu

In [13]:
train_0,val_0,test_0= normalized_data.filter(lambda x: x[-1] == 0.0).randomSplit([0.7,0.15,0.15], seed=42)
train_1,val_1,test_1= normalized_data.filter(lambda x: x[-1] == 1.0).randomSplit([0.7,0.15,0.15], seed=42)
train=train_0.union(train_1)
val=val_0.union(val_1)
test=test_0.union(test_1)

## Chuyển thành RDD LabeledPoint

In [14]:
train_data=train.map(lambda line: LabeledPoint(line[-1], Vectors.dense(line[0:-1])))
val_data=val.map(lambda line: LabeledPoint(line[-1], Vectors.dense(line[0:-1])))
test_data=test.map(lambda line: LabeledPoint(line[-1], Vectors.dense(line[0:-1])))


## Huấn luyện mô hình

- Sử dụng vòng lặp [20,50,100]
- Sử dụng regParam [0, 0.01, 0.1]
- Sử dụng thang đo recall trên label 1 để quyết định mô hình tốt nhất

In [15]:
iterations=[20,50,100]
regParams=[0, 0.01, 0.1]

best_recall=0.0
best_model=None
best_params=None
best_metrics=None

In [16]:
from pyspark.mllib.evaluation import MulticlassMetrics
for i in iterations:
    for j in regParams:
        print(f"Training with Iterations: {i}, RegParams {j}")
        model = LogisticRegressionWithLBFGS.train(train_data,iterations=i, regParam=j, intercept=True,numClasses=2)
        
        predictions = val_data.map(lambda p: (float(model.predict(p.features)),p.label))

        metrics = MulticlassMetrics(predictions)

        recall = metrics.recall(1.0)
        
        if recall > best_recall:
            best_recall = recall
            best_model = model
            best_metrics = metrics
            best_params = (i, j)
        print(f"Iterations: {i}, RegParams {j}, Recall: {recall:.4f}")

print(f"Best Recall: {best_recall:.4f} with Iterations: {best_params[0]}, RegParams {best_params[1]}")


Training with Iterations: 20, RegParams 0


C:\Program Files\Apache\Spark\python\pyspark\sql\context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Iterations: 20, RegParams 0, Recall: 0.6567
Training with Iterations: 20, RegParams 0.01
Iterations: 20, RegParams 0.01, Recall: 0.4925
Training with Iterations: 20, RegParams 0.1
Iterations: 20, RegParams 0.1, Recall: 0.2836
Training with Iterations: 50, RegParams 0
Iterations: 50, RegParams 0, Recall: 0.6716
Training with Iterations: 50, RegParams 0.01
Iterations: 50, RegParams 0.01, Recall: 0.4925
Training with Iterations: 50, RegParams 0.1
Iterations: 50, RegParams 0.1, Recall: 0.2836
Training with Iterations: 100, RegParams 0
Iterations: 100, RegParams 0, Recall: 0.6716
Training with Iterations: 100, RegParams 0.01
Iterations: 100, RegParams 0.01, Recall: 0.4925
Training with Iterations: 100, RegParams 0.1
Iterations: 100, RegParams 0.1, Recall: 0.2836
Best Recall: 0.6716 with Iterations: 50, RegParams 0


- Sau khi huấn luyện, mô hình tốt nhất có iteration = 50 và regParam = 0

## Đánh giá mô hình sau khi huấn luyện

In [17]:
print(f"Test Accuracy: {best_metrics.accuracy:.4f}")
print("Test Precision [0, 1]:", [best_metrics.precision(0.0), best_metrics.precision(1.0)])
print("Test Recall [0, 1]:", [best_metrics.recall(0.0), best_metrics.recall(1.0)])
print("Test F1 Score [0, 1]:", [best_metrics.fMeasure(0.0), best_metrics.fMeasure(1.0)])

Test Accuracy: 0.9994
Test Precision [0, 1]: [0.9994826207610178, 0.9375]
Test Recall [0, 1]: [0.9999294167470532, 0.6716417910447762]
Test F1 Score [0, 1]: [0.9997059688326964, 0.7826086956521741]


#### Nhận xét

- Độ chính xác tổng thể (Accuracy): 0.9991 - cực kỳ cao.
- Tuy nhiên, đi vào từng lớp:
    - Đối với lớp 0:
        - Đây là lớp chiếm đa số toàn bộ dữ liệu. 
        - Mô hình dự đoán rất tốt với precision, recall và F1 Score gần như tối đa. Kết quả gần như tuyệt đối.
    - Đối với lớp 1:
        - Lớp này chiếm phần nhỏ trong bộ dữ liệu (0.172%).
        - Số liệu đánh giá của mô hình thấp hơn lần lượt là precision 0.9375, recall 0.6716 và F1 Score 0.7826. Đối với bài toán này, việc recall của lớp 1 thấp (chỉ 0.6716) là vấn đề nghiêm trọng, bỏ xót nhiều trường hợp gian lận.

## Thực hiện dự đoán

In [18]:
predictions= test_data.map(lambda p: (float(best_model.predict(p.features)),p.label))

## Đánh giá mô hình sau khi dự đoán

In [19]:

metrics = MulticlassMetrics(predictions)

accuracy = metrics.accuracy
precision_1 = metrics.precision(1.0)
recall_1 = metrics.recall(1.0)
f1_score_1 = metrics.fMeasure(1.0)

precision_0 = metrics.precision(0.0)
recall_0 = metrics.recall(0.0)
f1_score_0 = metrics.fMeasure(0.0)

print(f"Test Accuracy: {accuracy:.4f}")
print("Test Precision [0, 1]:", [precision_0, precision_1])
print("Test Recall [0, 1]:", [recall_0, recall_1])
print("Test F1 Score [0, 1]:", [f1_score_0, f1_score_1])




C:\Program Files\Apache\Spark\python\pyspark\sql\context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


Test Accuracy: 0.9991
Test Precision [0, 1]: [0.9993167789662158, 0.813953488372093]
Test Recall [0, 1]: [0.9998114319387154, 0.546875]
Test F1 Score [0, 1]: [0.9995640442553994, 0.6542056074766355]


#### Nhận xét

- Kết quả của test thấp hơn so với khi huấn luyện. Để cải thiện hiệu suất ta có thể thử các phương pháp như undersampling lớp 0 hoặc oversampling lớp 1.

## Lưu kết quả

In [20]:
result=test_data.map(lambda p: (p.features, p.label, float(best_model.predict(p.features)))).toDF(["features", "label", "prediction"])

for row in result.take(5):
    print(row)

Row(features=DenseVector([0.0001, 0.9252, 0.7668, 0.8304, 0.207, 0.7855, 0.2927, 0.265, 0.7946, 0.4645, 0.5206, 0.2805, 0.6961, 0.4517, 0.6472, 0.4253, 0.4678, 0.7028, 0.6563, 0.6015, 0.578, 0.5521, 0.4727, 0.6784, 0.5147, 0.5959, 0.4499, 0.4296, 0.3134, 0.0]), label=0.0, prediction=0.0)
Row(features=DenseVector([0.0002, 0.9772, 0.769, 0.8464, 0.3107, 0.7644, 0.2632, 0.2652, 0.7866, 0.4634, 0.5097, 0.3653, 0.7527, 0.4288, 0.6553, 0.2757, 0.4307, 0.7337, 0.6154, 0.5512, 0.5782, 0.5609, 0.5145, 0.6647, 0.4135, 0.6119, 0.3696, 0.4172, 0.3133, 0.0003]), label=0.0, prediction=0.0)
Row(features=DenseVector([0.0002, 0.9769, 0.766, 0.8616, 0.3183, 0.7587, 0.2638, 0.2609, 0.7885, 0.4971, 0.5056, 0.2551, 0.7031, 0.346, 0.644, 0.3741, 0.4427, 0.7372, 0.6209, 0.5058, 0.5773, 0.5609, 0.5144, 0.6668, 0.4309, 0.592, 0.3637, 0.418, 0.3138, 0.0001]), label=0.0, prediction=0.0)
Row(features=DenseVector([0.0003, 0.9486, 0.7786, 0.8596, 0.276, 0.7696, 0.2604, 0.2718, 0.781, 0.4566, 0.5295, 0.3485, 0.7102,

In [21]:
result.coalesce(1).write.mode("overwrite").parquet("results")